In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree
from math import sqrt, ceil
from scipy.stats import linregress
    
import libraries as lib
import math 
    

In [2]:
#Lees de GPS data in

# Helper function to convert NMEA latitude/longitude to decimal degrees
def convert_to_decimal_degrees(value, direction):
    degrees = int(value) // 100
    minutes = value - (degrees * 100)
    decimal = degrees + (minutes / 60)
    
    if direction in ['S', 'W']:
        decimal = -decimal
    
    return decimal

# Function to read GPS data from a text file and convert to a DataFrame
def lees_gps_data(filepath):
    """Read GPS data from a text file and convert to a DataFrame."""
    timestamps = []
    latitudes = []
    lat_directions = []
    longitudes = []
    lon_directions = []
    hoogtes = []
    speeds = []
    
    with open(filepath, 'r') as file:
        for regel in file:
            parts = regel.split(',')
            if regel.startswith('$GPRMC'):
                # Extract time and speed from GPRMC sentence
                if len(parts) > 7:
                    timestamp = parts[1]
                    speed = parts[7]  # Speed in knots
                    timestamps.append(timestamp)
                    try:
                        speed_kmh = (float(speed) * 1.852) / 3.6  # Convert knots to m/s
                        speeds.append(speed_kmh)
                    except ValueError:
                        speeds.append(None)

            elif regel.startswith('$GPGGA'):
                # Extract latitude, longitude, and altitude from GPGGA sentence
                if len(parts) > 9:
                    latitude = parts[2]
                    lat_direction = parts[3]
                    longitude = parts[4] 
                    lon_direction = parts[5]
                    altitude = parts[9]
                    
                    # Append raw values for conversion later
                    try:
                        latitude = float(latitude)
                        latitudes.append(latitude)
                        lat_directions.append(lat_direction)
                    except ValueError:
                        latitudes.append(None)
                        lat_directions.append(None)
                    
                    try:
                        longitude = float(longitude)
                        longitudes.append(longitude)
                        lon_directions.append(lon_direction)
                    except ValueError:
                        longitudes.append(None)
                        lon_directions.append(None)
                    
                    # Convert altitude to float
                    try:
                        altitude = float(altitude)
                        hoogtes.append(altitude)
                    except ValueError:
                        hoogtes.append(None)
    
    # Check if all lists have the same length
    max_length = max(len(timestamps), len(latitudes), len(longitudes), len(hoogtes), len(speeds))
    
    # Make sure all lists have the same length by appending None to the shorter ones
    timestamps.extend([None] * (max_length - len(timestamps)))
    latitudes.extend([None] * (max_length - len(latitudes)))
    lat_directions.extend([None] * (max_length - len(lat_directions)))
    longitudes.extend([None] * (max_length - len(longitudes)))
    lon_directions.extend([None] * (max_length - len(lon_directions)))
    hoogtes.extend([None] * (max_length - len(hoogtes)))
    speeds.extend([None] * (max_length - len(speeds)))
    
    # Construct DataFrame
    data = {
        'timestamp': timestamps,
        'latitude': latitudes,
        'lat_direction': lat_directions,
        'longitude': longitudes,
        'lon_direction': lon_directions,
        'hoogte': hoogtes,
        'speed_m/s': speeds
    }
    
    df = pd.DataFrame(data)
    
    # Filter out rows where latitude or longitude are missing
    df = df.dropna(subset=['latitude', 'longitude']).reset_index(drop=True)
    
    # Convert latitude and longitude columns to decimal degrees
    df['latitude'] = df.apply(lambda row: convert_to_decimal_degrees(float(row['latitude']), row['lat_direction']), axis=1)
    df['longitude'] = df.apply(lambda row: convert_to_decimal_degrees(float(row['longitude']), row['lon_direction']), axis=1)
    
    return df

# Set file path
file_path = "/Users/willemheemskerk/Downloads/20230525_h2a_poging1/0002/Data voor hoogteprofiel.txt"

# Read GPS data
df = lees_gps_data(file_path)

# Print the resulting DataFrame
df


,timestamp,latitude,lat_direction,longitude,lon_direction,hoogte,speed_m/s
0,063448.30,43.771393,N,-0.042098,W,99.5,0.038583
1,063448.40,43.771393,N,-0.042098,W,99.5,0.026237
2,063448.50,43.771393,N,-0.042098,W,99.4,0.035497
3,063448.60,43.771393,N,-0.042098,W,99.5,0.073051
4,063448.70,43.771393,N,-0.042098,W,99.4,0.064820
...,...,...,...,...,...,...,...
46568,None,43.771202,N,-0.041941,W,96.0,NaN
46569,None,43.771202,N,-0.041941,W,96.0,NaN
46570,None,43.771202,N,-0.041941,W,96.1,NaN
46571,None,43.771202,N,-0.041941,W,96.1,NaN


In [3]:
#Functies voor het filter

def calculate_distance(lat1, lon1, lat2, lon2):
    return np.sqrt((lat2 - lat1) ** 2 + (lon2 - lon1) ** 2)

def find_nearest_index(df, target_lon, target_lat):
    distances = df.apply(lambda row: calculate_distance(row['latitude'], row['longitude'], target_lat, target_lon), axis=1)
    return distances.idxmin()

def filter_track_data(df, start_lon, start_lat, end_lon, end_lat):
    start_index = find_nearest_index(df, start_lon, start_lat)
    end_index = find_nearest_index(df, end_lon, end_lat)
    
    # Zorg ervoor dat de start_index voor de end_index komt
    if start_index > end_index:
        start_index, end_index = end_index, start_index
    
    return df.iloc[start_index:end_index + 1]


In [4]:
#Filter vanaf het moment dat de auto de baan opgaat en wanneer die er weer afgaat.

# Definieer de coördinaten voor het start- en eindpunt
start_lon = -0.0446
start_lat = 0.00095 + 4.3771e1
end_lon = -0.039925
end_lat = 0.000300 + 4.3771e1

# Veronderstel dat 'df' je DataFrame is met de relevante data
filtered_df = filter_track_data(df, start_lon, start_lat, end_lon, end_lat)

# Print het gefilterde DataFrame om te controleren
filtered_df
#43.767914, -0.035855

,timestamp,latitude,lat_direction,longitude,lon_direction,hoogte,speed_m/s
22340,071203.60,43.771951,N,-0.044597,W,102.0,5.597156
22341,071203.70,43.771954,N,-0.044603,W,102.0,5.624421
22342,071203.80,43.771955,N,-0.044609,W,102.0,5.633167
22343,071203.90,43.771958,N,-0.044616,W,102.0,5.650143
22344,071204.00,43.771960,N,-0.044622,W,102.0,5.655288
...,...,...,...,...,...,...,...
42943,074720.40,43.771037,N,-0.040149,W,98.1,8.270209
42944,074720.50,43.771032,N,-0.040142,W,98.1,8.298503
42945,074720.60,43.771027,N,-0.040135,W,98.1,8.276382
42946,074720.70,43.771022,N,-0.040127,W,98.1,8.287700


In [5]:
#Plot om te checken of het is gelukt en ref met maps
#Het is gelukt.

%matplotlib qt

# Definieer een functie voor een 2D-plot
def plot_2d_height_profile(latitudes, longitudes, hoogtes):
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(longitudes, latitudes, c=hoogtes, cmap='viridis', s=10, alpha=0.8)
    plt.scatter(-0.039853, 43.770328, color='red', s=100, marker='o', label='Specifiek punt')
    plt.colorbar(scatter, label='Hoogte (m)')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('2D Hoogteprofiel')
    plt.grid(True)
    plt.show()

# Voorbeeld dataframe (hier kun je je eigen gefilterde data inladen)
# Gebruik de functie met de gefilterde data
plot_2d_height_profile(filtered_df['latitude'], filtered_df['longitude'], filtered_df['hoogte'])


In [14]:
#Filter voor de rechte stukken.

# Set initial flag and empty list
gevonden = False
gevonden2 = False
rechtestuk_rijen_hoogte = []
bochten_rijen_hoogte = []

# Define the coordinates for the two rectangular areas
lat_min, lat_max = 7.6e-5 + 4.37724e1, 8.25e-5 + 4.37724e1
lon_min, lon_max = -0.04226, -0.04220

lat_min2, lat_max2 = 73.15e-5 + 4.377e1, 74e-5 + 4.377e1
lon_min2, lon_max2 = -0.03983, -0.03979

lat_min3, lat_max3 = 73.15e-5 + 4.377e1, 74e-5 + 4.377e1
lon_min3, lon_max3 = -0.03983, -0.03979

lat_min4, lat_max4 =  (36 * 1e-6) + (4.37704e1), (40 * 1e-6) + (4.37704e1)
lon_min4, lon_max4 = -0.04034, -0.04029


#Iterate over each row in the DataFrame (replace df with your actual DataFrame)
for i in range(len(filtered_df)):
    lat = filtered_df['latitude'].iloc[i]  # Latitude of the i-th row
    lon = filtered_df['longitude'].iloc[i]  # Longitude of the i-th row

    # Check if the point is within the first box
    if lat_min <= lat <= lat_max and lon_min <= lon <= lon_max:
        if not gevonden:  # Append only the first time a point is found in the first box
            rechtestuk_rijen_hoogte.append(i)
            gevonden = True  # Set 'gevonden' flag to True

    # Check if the point is within the second box after finding a point in the first box
    if gevonden and lat_min2 <= lat <= lat_max2 and lon_min2 <= lon <= lon_max2:
        rechtestuk_rijen_hoogte.append(i)
        gevonden = False  # Reset 'gevonden' flag to False after processing the second box
        
    #Optionally, you can print out each step to debug

    

# Print the results
print("Indices of points in the two boxes:", rechtestuk_rijen_hoogte)


Indices of points in the two boxes: [962, 1282, 3057, 3356, 5065, 5370, 7067, 7372, 9237, 9561, 11379, 11698, 13581, 13907, 15783, 16105, 18038, 18368, 20316]


In [18]:
# Reset de lijst en flags
bochten_rijen_hoogte = []
gevonden3 = False  # Flag voor rechthoek 3
gevonden4 = False  # Flag voor rechthoek 4

for j in range(len(filtered_df)):
    lat = filtered_df['latitude'].iloc[j]
    lon = filtered_df['longitude'].iloc[j]
    
    # Controleer op de eerste rechthoek (lat_min4/lon_min4)
    if lat_min4 <= lat <= lat_max4 and lon_min4 <= lon <= lon_max4:
        if not gevonden4:  # Alleen de eerste keer toevoegen
            bochten_rijen_hoogte.append(j)
            gevonden4 = True  # Set de flag voor rechthoek 4

    # Controleer op de tweede rechthoek (lat_min3/lon_min3)
    if lat_min3 <= lat <= lat_max3 and lon_min3 <= lon <= lon_max3:
        if not gevonden3:  # Alleen de eerste keer toevoegen
            bochten_rijen_hoogte.append(j)
            gevonden3 = True  # Set de flag voor rechthoek 3
            
print(f"Checking point {j}: lat={lat}, lon={lon}")
print(f"Rechthoek 3: {lat_min3 <= lat <= lat_max3 and lon_min3 <= lon <= lon_max3}")
print(f"Rechthoek 4: {lat_min4 <= lat <= lat_max4 and lon_min4 <= lon <= lon_max4}")

print("Indices van punten in bochten (rechthoek 3 en 4):", bochten_rijen_hoogte)


Checking point 20607: lat=43.771016833333334, lon=-0.04012
Rechthoek 3: False
Rechthoek 4: False
Indices van punten in bochten (rechthoek 3 en 4): [1282, 1409]


In [58]:
# Imports
import matplotlib.pyplot as plt
gebieden = {
    "rechthoek1": [7.6e-5 + 4.37724e1, 8.25e-5 + 4.37724e1, -0.04226, -0.04220],
    "rechthoek2": [73.15e-5 + 4.377e1, 74e-5 + 4.377e1, -0.03983, -0.03979],
    "rechthoek3": [(36 * 1e-6) + (4.37704e1), (40 * 1e-6) + (4.37704e1), -0.04034, -0.04029],
}
def plot_hoogteprofiel_2d(latitudes, longitudes, hoogtes, rechtestuk_indices, bochten_indices):
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Hoofdscatterplot voor alle data
    scatter = ax.scatter(longitudes, latitudes, c=hoogtes, cmap='viridis', marker='o', alpha=0.5, label="Alle data")
    
    # Plot rechte stukken
    ax.scatter(
        longitudes.iloc[rechtestuk_indices], 
        latitudes.iloc[rechtestuk_indices], 
        color='blue', 
        label='Rechte stukken', 
        marker='x'
    )
    
    # Plot bochten
    ax.scatter(
        longitudes.iloc[bochten_indices], 
        latitudes.iloc[bochten_indices], 
        color='red', 
        label='Bochten', 
        marker='^'
    )

    # Titel en labels
    ax.set_title('Hoogteprofiel van racebaan Nogaro', fontsize=16)
    ax.set_xlabel('Longitude', fontsize=14)
    ax.set_ylabel('Latitude', fontsize=14)

    # Colorbar toevoegen
    cbar = fig.colorbar(scatter, ax=ax)
    cbar.set_label('Hoogte boven zeeniveau (m)', fontsize=12)
    
    # Plot rechthoeken
    for key, (lat_min, lat_max, lon_min, lon_max) in gebieden.items():
        ax.plot(
            [lon_min, lon_max, lon_max, lon_min, lon_min],
            [lat_min, lat_min, lat_max, lat_max, lat_min],
            linestyle='--', linewidth=2, label=f"Boundary {key}"
        )

    # Legenda en grid
    ax.legend()
    plt.grid(True)
    plt.show()
    for i in range(len(latitudes)):
        lat = latitudes.iloc[i]
        lon = longitudes.iloc[i]
        #plt.text(lon, lat, str(i), fontsize=8, color='black')

# Plot de resultaten
plot_hoogteprofiel_2d(
    filtered_df['latitude'], 
    filtered_df['longitude'], 
    filtered_df['hoogte'], 
    rechtestuk_rijen_hoogte, 
    bochten_rijen_hoogte
)


In [59]:
#Check afstand tussen begin en eind rechte stuk
import math
lat_min3, lat_max3 = 73.15e-5 + 4.377e1, 74e-5 + 4.377e1
lon_min3, lon_max3 = -0.03983, -0.03979

lat_min4, lat_max4 =  (36 * 1e-6) + (4.37704e1), (40 * 1e-6) + (4.37704e1)
lon_min4, lon_max4 = -0.04034, -0.04029
# Jouw haversine functie
def haversine(lat3, lon3, lat4, lon2, radius=6371000):  # Radius in meters
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Compute differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Distance in meters
    distance = radius * c
    return distance

# Omzetten van 'latitude' en 'longitude' naar correcte waarden (rekening houdend met N/S en E/W)
def correct_coordinates(lat, lat_dir, lon, lon_dir):
    if lat_dir == 'S':
        lat = -lat
    if lon_dir == 'W':
        lon = -lon
    return lat, lon

# Functie om de afstand te berekenen tussen twee rijen in de DataFrame
def calculate_distance_between_rows(df, row_index1, row_index2):
    # Haal de coördinaten en richting op voor de twee rijen
    lat1, lon1 = correct_coordinates(
        df.loc[row_index1, 'latitude'], df.loc[row_index1, 'lat_direction'],
        df.loc[row_index1, 'longitude'], df.loc[row_index1, 'lon_direction']
    )
    lat2, lon2 = correct_coordinates(
        df.loc[row_index2, 'latitude'], df.loc[row_index2, 'lat_direction'],
        df.loc[row_index2, 'longitude'], df.loc[row_index2, 'lon_direction']
    )
    
    # Bereken de afstand met de haversine functie
    distance = haversine(lat1, lon1, lat2, lon2)
    return distance

# Voorbeeld van hoe de functie te gebruiken
# Stel dat filtered_df al je DataFrame is
row_index1 = 1282+22340  # Vervang dit door de index van de eerste rij
row_index2 = 1409+22340  # Vervang dit door de index van de tweede rij

# Bereken de afstand tussen de twee punten
afstand = calculate_distance_between_rows(filtered_df, row_index1, row_index2)
print(f"De afstand tussen rij {row_index1} en rij {row_index2} is: {afstand} meter")



UnboundLocalError: local variable 'lat1' referenced before assignment

In [55]:

#Bereken delta x en delta h overal en laat deze in een aparte kolom sommeren
# Jouw haversine functie
def haversine(lat1, lon1, lat2, lon2, radius=6371000):  # Radius in meters
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Compute differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Distance in meters
    distance = radius * c
    return distance

# Omzetten van 'latitude' en 'longitude' naar correcte waarden (rekening houdend met N/S en E/W)
def correct_coordinates(lat, lat_dir, lon, lon_dir):
    if lat_dir == 'S':
        lat = -lat
    if lon_dir == 'W':
        lon = -lon
    return lat, lon

# Functie om de afstand te berekenen tussen twee rijen in de DataFrame
def calculate_distance_between_rows(filtered_df, row_index1, row_index2):
    # Haal de coördinaten en richting op voor de twee rijen
    lat1, lon1 = correct_coordinates(
        filtered_df.loc[row_index1, 'latitude'], filtered_df.loc[row_index1, 'lat_direction'],
        filtered_df.loc[row_index1, 'longitude'], filtered_df.loc[row_index1, 'lon_direction']
    )
    lat2, lon2 = correct_coordinates(
        filtered_df.loc[row_index2, 'latitude'], filtered_df.loc[row_index2, 'lat_direction'],
        filtered_df.loc[row_index2, 'longitude'], filtered_df.loc[row_index2, 'lon_direction']
    )
    
    # Bereken de afstand met de haversine functie
    distance = haversine(lat1, lon1, lat2, lon2)
    return distance

# Functie om hoogteverschil te berekenen tussen twee rijen
def calculate_height_difference(df, row_index1, row_index2):
    hoogte1 = df.loc[row_index1, 'hoogte']
    hoogte2 = df.loc[row_index2, 'hoogte']
    return hoogte2 - hoogte1

# Bereken de afstand en hoogteverschil voor elke rij van 23302 tot en met 23622
rows = range(1282+22340, 1409+22340)
afstanden = []
hoogteverschillen = []

for i in range(len(rows) - 1):
    row_index1 = rows[i]
    row_index2 = rows[i + 1]
    
    # Bereken afstand en hoogteverschil
    afstand = calculate_distance_between_rows(filtered_df, row_index1, row_index2)
    hoogteverschil = calculate_height_difference(filtered_df, row_index1, row_index2)
    
    afstanden.append(afstand)
    hoogteverschillen.append(hoogteverschil)

# Maak een DataFrame met de resultaten
hoogte_vs_afstand = pd.DataFrame({
    'afstand': afstanden,
    'hoogteverschil': hoogteverschillen
})

# Voeg cumulatieve afstand en cumulatief hoogteverschil toe
hoogte_vs_afstand['cumulatieve_afstand'] = hoogte_vs_afstand['afstand'].cumsum()
hoogte_vs_afstand['cumulatief_hoogteverschil'] = hoogte_vs_afstand['hoogteverschil'].cumsum()

# Print de DataFrame
hoogte_vs_afstand


,afstand,hoogteverschil,cumulatieve_afstand,cumulatief_hoogteverschil
0,0.800092,0.0,0.800092,0.0
1,1.587040,-0.1,2.387132,-0.1
2,0.778652,0.0,3.165784,-0.1
3,0.794589,0.0,3.960373,-0.1
4,0.787912,0.0,4.748284,-0.1
...,...,...,...,...
121,0.496189,0.0,85.944645,0.1
122,0.488001,0.0,86.432645,0.1
123,0.488001,0.0,86.920646,0.1
124,0.502905,0.0,87.423551,0.1


In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap

# Data for the plot
hoogte_vs_afstand = pd.DataFrame(hoogte_vs_afstand)
x = hoogte_vs_afstand['cumulatieve_afstand'].loc[1:]
y = hoogte_vs_afstand['cumulatief_hoogteverschil'].loc[1:]

# Normalize data (optional, for numerical stability)
x_scaled = (x - x.min()) / (x.max() - x.min())
y_scaled = (y - y.min()) / (y.max() - y.min())

# Define degrees for the fits
degrees = np.linspace(5, len(afstanden), 2, dtype='int')

# Get a colormap for consistent color generation
cmap = get_cmap("viridis", len(degrees))  # Using 'viridis', adjust for other schemes

# Plot the scatter points
plt.figure(figsize=(10, 6))
plt.scatter(x, y, label='Data punten', color='black', s=15)

# Loop over each degree and plot the fits with unique colors and labels
for i, degree in enumerate(degrees):
    try:
        # Calculate polynomial fit
        coefficients = np.polyfit(x_scaled, y_scaled, degree)
        polynomial = np.poly1d(coefficients)
        y_fit = polynomial(x_scaled)
        
        # Generate unique color and label
        color = cmap(i)
        label = f'Fit (graad {degree})'
        
        # Plot the fit
        plt.plot(
            x, 
            y_fit * (y.max() - y.min()) + y.min(),  # Rescale y_fit to original scale
            color=color, linestyle='-', linewidth=2, label=label
        )
    except np.linalg.LinAlgError as e:
        print(f"Fit mislukte voor graad {degree}: {e}")

# Labels and title
plt.xlabel('Cumulatieve Afstand (m)', fontsize=14)
plt.ylabel('Cumulatief Hoogteverschil (m)', fontsize=14)
plt.title('Cumulatief Hoogteverschil vs. Cumulatieve Afstand', fontsize=16)
plt.grid(visible=True, linestyle='--', alpha=0.7)
plt.legend(fontsize=12, loc='upper left')
plt.tight_layout()

# Show the plot
plt.show()


/var/folders/4x/syjxsxb92jb63fywzwb8mgvh0000gn/T/ipykernel_77659/2415400029.py:19: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = get_cmap("viridis", len(degrees))  # Using 'viridis', adjust for other schemes
/Users/willemheemskerk/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)


In [57]:
import numpy as np
import matplotlib.pyplot as plt

# Zorg ervoor dat hoogte_vs_afstand een DataFrame is met 'cumulatieve_afstand' en 'cumulatief_hoogteverschil'
hoogte_vs_afstand = pd.DataFrame(hoogte_vs_afstand)

# Data voor de plot
x = hoogte_vs_afstand['cumulatieve_afstand']
y = hoogte_vs_afstand['cumulatief_hoogteverschil']

# Definieer de graden van de polynomen die je wilt passen
degrees = [2, 5, 7]  # Pas meerdere graden aan
fit_results = {}

# Plot de scatterpunten
plt.figure(figsize=(12, 8))
plt.scatter(x, y, label='Data punten', color='black', s=15)

# Pas meerdere fits aan en plot
for degree in degrees:
    coefficients = np.polyfit(x, y, degree)
    polynomial = np.poly1d(coefficients)
    y_fit = polynomial(x)
    
    # Bewaar de resultaten voor latere analyse
    fit_results[degree] = {"coefficients": coefficients, "polynomial": polynomial}
    
    # Plot de fitlijn
    plt.plot(x, y_fit, label=f'Beste fit (graad {degree})')
    
    # Genereer de formule van de polynoomfit
    fit_equation = "y = " + " + ".join(
        [f"{coef:.2e}x^{degree - i}" if degree - i > 0 else f"{coef:.2e}" 
         for i, coef in enumerate(coefficients)]
    )
    # Print de formule op de plot
    plt.text(
        0.05 * max(x),
        (0.85 - 0.1 * degrees.index(degree)) * max(y),  # Verschillende posities voor elke formule
        f"Graad {degree}: {fit_equation}",
        fontsize=10,
        bbox=dict(facecolor='white', alpha=0.6)
    )

# Labels en titel
plt.xlabel('Cumulatieve Afstand (m)', fontsize=14)
plt.ylabel('Cumulatief Hoogteverschil (m)', fontsize=14)
plt.title('Cumulatief Hoogteverschil vs. Cumulatieve Afstand met Meerdere Fits', fontsize=16)
plt.grid(visible=True, linestyle='--', alpha=0.7)
plt.legend(fontsize=12, loc='lower left')
plt.tight_layout()

# Toon de plot
plt.show()

# Print de fitvergelijkingen in de console
for degree, result in fit_results.items():
    coefficients = result["coefficients"]
    fit_equation = "y = " + " + ".join(
        [f"{coef:.2e}x^{degree - i}" if degree - i > 0 else f"{coef:.2e}" 
         for i, coef in enumerate(coefficients)]
    )
    print(f"Graad {degree} fitvergelijking: {fit_equation}")


Graad 2 fitvergelijking: y = 1.78e-05x^2 + 1.81e-03x^1 + -1.74e-01
Graad 5 fitvergelijking: y = 7.11e-10x^5 + -8.25e-08x^4 + -3.99e-06x^3 + 7.87e-04x^2 + -2.36e-02x^1 + 5.06e-03
Graad 7 fitvergelijking: y = 4.61e-12x^7 + -1.48e-09x^6 + 1.91e-07x^5 + -1.24e-05x^4 + 4.21e-04x^3 + -6.60e-03x^2 + 3.14e-02x^1 + -1.08e-01
